In [1]:
import pandas as pd
import numpy as np
import gc 
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%%time
# 파일 로드
train_df = pd.read_csv('./data/train.csv')
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'], format = '%Y-%m-%d %H:%M:%S')
weather_train_df = pd.read_csv('./data/weather_train.csv')


test_df = pd.read_csv('./data/test.csv')
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'], format = '%Y-%m-%d %H:%M:%S')

weather_test_df = pd.read_csv('./data/weather_test.csv')
building_meta_df = pd.read_csv('./data/building_metadata.csv')
sample_submission = pd.read_csv('./data/sample_submission.csv')

Wall time: 26.7 s


In [3]:
print(train_df.shape)
print(weather_train_df.shape)
print(weather_test_df.shape)
print(building_meta_df.shape)

print(test_df.shape)

(20216100, 4)
(139773, 9)
(277243, 9)
(1449, 6)
(41697600, 4)


In [4]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [5]:
# 메모리 절약
train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)

weather_train_df = reduce_mem_usage(weather_train_df)
weather_test_df = reduce_mem_usage(weather_test_df)
building_meta_df = reduce_mem_usage(building_meta_df)

Mem. usage decreased to 289.19 Mb (53.1% reduction)
Mem. usage decreased to 596.49 Mb (53.1% reduction)
Mem. usage decreased to  3.07 Mb (68.1% reduction)
Mem. usage decreased to  6.08 Mb (68.1% reduction)
Mem. usage decreased to  0.03 Mb (60.3% reduction)


In [6]:
# train_df -- timestamp : 월 , 주, 일

train_df['meter_reading_log1p'] = np.log1p(train_df['meter_reading'])

train_df['date'] = train_df['timestamp'].dt.date
train_df['hour'] = train_df['timestamp'].dt.hour
train_df['weekend'] = train_df['timestamp'].dt.weekday
train_df['month'] = train_df['timestamp'].dt.month
train_df['dayofweek'] = train_df['timestamp'].dt.dayofweek


test_df['date'] = test_df['timestamp'].dt.date
test_df['hour'] = test_df['timestamp'].dt.hour
test_df['weekend'] = test_df['timestamp'].dt.weekday
test_df['month'] = test_df['timestamp'].dt.month
test_df['dayofweek'] = test_df['timestamp'].dt.dayofweek

In [7]:
weather_train_df.isnull().sum()

site_id                   0
timestamp                 0
air_temperature          55
cloud_coverage        69173
dew_temperature         113
precip_depth_1_hr     50289
sea_level_pressure    10618
wind_direction         6268
wind_speed              304
dtype: int64

In [8]:
weather_train_df.head()
weather_train_df = weather_train_df.groupby('site_id').apply\
                    (lambda group : group.interpolate(limit_direction='both'))
weather_test_df = weather_test_df.groupby('site_id').apply\
                    (lambda group : group.interpolate(limit_direction='both'))

In [9]:
weather_train_df

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.000000,6.0,20.000000,-1.0,1019.5,0.0,0.000000
1,0,2016-01-01 01:00:00,24.406250,4.0,21.093750,-1.0,1020.0,70.0,1.500000
2,0,2016-01-01 02:00:00,22.796875,2.0,21.093750,0.0,1020.0,0.0,0.000000
3,0,2016-01-01 03:00:00,21.093750,2.0,20.593750,0.0,1020.0,0.0,0.000000
4,0,2016-01-01 04:00:00,20.000000,2.0,20.000000,-1.0,1020.0,250.0,2.599609
...,...,...,...,...,...,...,...,...,...
139768,15,2016-12-31 19:00:00,3.000000,1.5,-8.000000,-1.0,1007.5,180.0,5.699219
139769,15,2016-12-31 20:00:00,2.800781,2.0,-8.898438,-1.0,1007.5,180.0,7.699219
139770,15,2016-12-31 21:00:00,2.800781,2.0,-7.199219,-1.0,1007.5,180.0,5.101562
139771,15,2016-12-31 22:00:00,2.199219,2.0,-6.699219,-1.0,1008.0,170.0,4.601562


In [10]:
# site 별로 최근 3일간의 날씨를 rolling 하기
def add_lag_feature(weather_df, window=3):
    group_df = weather_df.groupby('site_id')
    cols = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
    rolled = group_df[cols].rolling(window=window, min_periods=0)
    lag_mean = rolled.mean().reset_index().astype(np.float16)
    lag_max = rolled.max().reset_index().astype(np.float16)
    lag_min = rolled.min().reset_index().astype(np.float16)
    lag_std = rolled.std().reset_index().astype(np.float16)
    for col in cols:
        weather_df[f'{col}_mean_lag{window}'] = lag_mean[col]
        #weather_df[f'{col}_max_lag{window}'] = lag_max[col]
        #weather_df[f'{col}_min_lag{window}'] = lag_min[col]
        #weather_df[f'{col}_std_lag{window}'] = lag_std[col]

In [11]:
add_lag_feature(weather_train_df, window=3)
add_lag_feature(weather_train_df, window=72) #24*3일
add_lag_feature(weather_test_df, window=3)
add_lag_feature(weather_test_df, window=72)

In [12]:
train_df = pd.merge(train_df,building_meta_df, on= ['building_id'],how='left')
test_df = pd.merge(test_df,building_meta_df, on= ['building_id'],how='left')

In [13]:
weather_train_df['timestamp'] = pd.to_datetime(weather_train_df['timestamp'], format = '%Y-%m-%d %H:%M:%S')
weather_test_df['timestamp'] = pd.to_datetime(weather_test_df['timestamp'], format = '%Y-%m-%d %H:%M:%S')

train_df = pd.merge(train_df,weather_train_df, on= ['site_id','timestamp'],how='left')
test_df = pd.merge(test_df,weather_test_df, on= ['site_id','timestamp'],how='left')

In [14]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [15]:
train_df['primary_use'].value_counts()/len(train_df)

Education                        0.403911
Office                           0.217394
Entertainment/public assembly    0.112035
Lodging/residential              0.106173
Public services                  0.082253
Healthcare                       0.019713
Other                            0.011982
Parking                          0.010576
Manufacturing/industrial         0.006218
Food sales and service           0.005644
Retail                           0.005573
Warehouse/storage                0.005533
Services                         0.004774
Technology/science               0.003840
Utility                          0.002780
Religious worship                0.001601
Name: primary_use, dtype: float64

In [16]:
test_df['primary_use'].value_counts()/len(test_df)

Education                        0.402941
Office                           0.215966
Entertainment/public assembly    0.113025
Lodging/residential              0.107983
Public services                  0.081933
Healthcare                       0.019328
Other                            0.012185
Parking                          0.010504
Manufacturing/industrial         0.006303
Retail                           0.005462
Warehouse/storage                0.005462
Food sales and service           0.005462
Services                         0.004622
Technology/science               0.004202
Utility                          0.002941
Religious worship                0.001681
Name: primary_use, dtype: float64

In [17]:
#import category_encoders as ce
#encoder = ce.OneHotEncoder()
#train_pri_use = encoder.fit_transform(train_df['primary_use'])
#test_pri_use = encoder.fit_transform(test_df['primary_use'])
#train_df = pd.concat([train_df,train_pri_use], axis=1)
#test_df = pd.concat([test_df,test_pri_use], axis=1)

In [18]:
#Encoidng variables
le = LabelEncoder()
# train_df['primary_use'] = train_df['primary_use'].astype(str)
train_df['primary_use'] = le.fit_transform(train_df['primary_use']).astype(np.int8)

# test_df['primary_use'] = test_df['primary_use'].astype(str)
test_df['primary_use'] = le.fit_transform(test_df['primary_use']).astype(np.int8)

In [19]:

#train_df.drop('primary_use', axis=1, inplace=True)
#test_df.drop('primary_use', axis=1, inplace=True)

In [20]:
train_df.columns

Index(['building_id', 'meter', 'timestamp', 'meter_reading',
       'meter_reading_log1p', 'date', 'hour', 'weekend', 'month', 'dayofweek',
       'site_id', 'primary_use', 'square_feet', 'year_built', 'floor_count',
       'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
       'wind_speed', 'air_temperature_mean_lag3', 'cloud_coverage_mean_lag3',
       'dew_temperature_mean_lag3', 'precip_depth_1_hr_mean_lag3',
       'sea_level_pressure_mean_lag3', 'wind_direction_mean_lag3',
       'wind_speed_mean_lag3', 'air_temperature_mean_lag72',
       'cloud_coverage_mean_lag72', 'dew_temperature_mean_lag72',
       'precip_depth_1_hr_mean_lag72', 'sea_level_pressure_mean_lag72',
       'wind_direction_mean_lag72', 'wind_speed_mean_lag72'],
      dtype='object')

In [21]:
train_df

,building_id,meter,timestamp,meter_reading,meter_reading_log1p,date,hour,weekend,month,dayofweek,...,sea_level_pressure_mean_lag3,wind_direction_mean_lag3,wind_speed_mean_lag3,air_temperature_mean_lag72,cloud_coverage_mean_lag72,dew_temperature_mean_lag72,precip_depth_1_hr_mean_lag72,sea_level_pressure_mean_lag72,wind_direction_mean_lag72,wind_speed_mean_lag72
0,0,0,2016-01-01 00:00:00,0.000000,0.000000,2016-01-01,0,4,1,4,...,1019.5,0.000,0.000000,25.00000,6.000,20.000000,-1.000000,1019.5,0.000,0.0000
1,1,0,2016-01-01 00:00:00,0.000000,0.000000,2016-01-01,0,4,1,4,...,1019.5,0.000,0.000000,25.00000,6.000,20.000000,-1.000000,1019.5,0.000,0.0000
2,2,0,2016-01-01 00:00:00,0.000000,0.000000,2016-01-01,0,4,1,4,...,1019.5,0.000,0.000000,25.00000,6.000,20.000000,-1.000000,1019.5,0.000,0.0000
3,3,0,2016-01-01 00:00:00,0.000000,0.000000,2016-01-01,0,4,1,4,...,1019.5,0.000,0.000000,25.00000,6.000,20.000000,-1.000000,1019.5,0.000,0.0000
4,4,0,2016-01-01 00:00:00,0.000000,0.000000,2016-01-01,0,4,1,4,...,1019.5,0.000,0.000000,25.00000,6.000,20.000000,-1.000000,1019.5,0.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20216095,1444,0,2016-12-31 23:00:00,8.750000,2.277267,2016-12-31,23,5,12,5,...,1008.0,176.625,6.167969,-1.31543,1.375,-5.234375,-0.644531,1008.0,210.875,5.3125
20216096,1445,0,2016-12-31 23:00:00,4.825000,1.762159,2016-12-31,23,5,12,5,...,1008.0,176.625,6.167969,-1.31543,1.375,-5.234375,-0.644531,1008.0,210.875,5.3125
20216097,1446,0,2016-12-31 23:00:00,0.000000,0.000000,2016-12-31,23,5,12,5,...,1008.0,176.625,6.167969,-1.31543,1.375,-5.234375,-0.644531,1008.0,210.875,5.3125
20216098,1447,0,2016-12-31 23:00:00,159.574997,5.078761,2016-12-31,23,5,12,5,...,1008.0,176.625,6.167969,-1.31543,1.375,-5.234375,-0.644531,1008.0,210.875,5.3125


In [22]:
train_df['age'] = train_df['year_built'].max()-train_df['year_built']+1
test_df['age'] = test_df['year_built'].max() - test_df['year_built'] + 1

In [23]:
train_df['floor_count'] = train_df['floor_count'].fillna(-999).astype(np.int16)
test_df['floor_count'] = test_df['floor_count'].fillna(-999).astype(np.int16)

train_df['year_built'] = train_df['year_built'].fillna(-999).astype(np.int16)
test_df['year_built'] = test_df['year_built'].fillna(-999).astype(np.int16)

train_df['age'] = train_df['age'].fillna(-999).astype(np.int16)
test_df['age'] = test_df['age'].fillna(-999).astype(np.int16)

train_df['cloud_coverage'] = train_df['cloud_coverage'].fillna(-999).astype(np.int16)
test_df['cloud_coverage'] = test_df['cloud_coverage'].fillna(-999).astype(np.int16) 


In [24]:
holidays = ["2016-01-01", "2016-01-18", "2016-02-15", "2016-05-30", "2016-07-04",
                "2016-09-05", "2016-10-10", "2016-11-11", "2016-11-24", "2016-12-26",
                "2017-01-01", "2017-01-16", "2017-02-20", "2017-05-29", "2017-07-04",
                "2017-09-04", "2017-10-09", "2017-11-10", "2017-11-23", "2017-12-25",
                "2018-01-01", "2018-01-15", "2018-02-19", "2018-05-28", "2018-07-04",
                "2018-09-03", "2018-10-08", "2018-11-12", "2018-11-22", "2018-12-25",
                "2019-01-01"]

train_df["is_holiday"] = (train_df.timestamp.dt.date.astype("str").isin(holidays)).astype(int)
test_df["is_holiday"] = (test_df.timestamp.dt.date.astype("str").isin(holidays)).astype(int)

In [25]:
drop_cols = ['date',"precip_depth_1_hr", "sea_level_pressure", "wind_direction", "wind_speed","timestamp"]
target = train_df["meter_reading_log1p"]
del train_df["meter_reading"], train_df['meter_reading_log1p']
train_df = train_df.drop(drop_cols, axis=1)
drop_cols += ["row_id"]
# drop_cols.remove('date')
test_df = test_df.drop(drop_cols, axis=1)

In [26]:
import lightgbm as lgb
from sklearn.model_selection import KFold
import xgboost as xgb
# lightbgm
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2'},
    'subsample': 0.2,
    'learning_rate': 0.08,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.9,
    'alpha': 0.1,
    'lambda': 0.1,
#     'n_jobs' :2 
}
#xgboost
xgb_params = {
    'colsample_bytree': 0.8,                 
    'learning_rate': 0.05,
    'max_depth': 10,
    'subsample': 0.8,
    'reg_alpha' :0.15,
    'reg_lamdba' : 0.85,
    'tree_method': 'gpu_hist',
    'missing' : -999,    
    'objective': 'reg:squarederror',
#     'n_jobs' : 4
}

In [27]:
train_df['floor_count'].value_counts()

-999    16709167
 1        934303
 2        691052
 4        351938
 3        331688
 6        296598
 5        253856
 8        175258
 7        131435
 9         86416
 11        61465
 13        43911
 10        34880
 19        26343
 21        26343
 26        26321
 12        17559
 16         8784
 14         8783
Name: floor_count, dtype: int64

In [28]:
train_df

,building_id,meter,hour,weekend,month,dayofweek,site_id,primary_use,square_feet,year_built,...,wind_speed_mean_lag3,air_temperature_mean_lag72,cloud_coverage_mean_lag72,dew_temperature_mean_lag72,precip_depth_1_hr_mean_lag72,sea_level_pressure_mean_lag72,wind_direction_mean_lag72,wind_speed_mean_lag72,age,is_holiday
0,0,0,0,4,1,4,0,0,7432,2008,...,0.000000,25.00000,6.000,20.000000,-1.000000,1019.5,0.000,0.0000,10,1
1,1,0,0,4,1,4,0,0,2720,2004,...,0.000000,25.00000,6.000,20.000000,-1.000000,1019.5,0.000,0.0000,14,1
2,2,0,0,4,1,4,0,0,5376,1991,...,0.000000,25.00000,6.000,20.000000,-1.000000,1019.5,0.000,0.0000,27,1
3,3,0,0,4,1,4,0,0,23685,2002,...,0.000000,25.00000,6.000,20.000000,-1.000000,1019.5,0.000,0.0000,16,1
4,4,0,0,4,1,4,0,0,116607,1975,...,0.000000,25.00000,6.000,20.000000,-1.000000,1019.5,0.000,0.0000,43,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20216095,1444,0,23,5,12,5,15,1,19619,1914,...,6.167969,-1.31543,1.375,-5.234375,-0.644531,1008.0,210.875,5.3125,104,0
20216096,1445,0,23,5,12,5,15,0,4298,-999,...,6.167969,-1.31543,1.375,-5.234375,-0.644531,1008.0,210.875,5.3125,-999,0
20216097,1446,0,23,5,12,5,15,1,11265,1997,...,6.167969,-1.31543,1.375,-5.234375,-0.644531,1008.0,210.875,5.3125,21,0
20216098,1447,0,23,5,12,5,15,4,29775,2001,...,6.167969,-1.31543,1.375,-5.234375,-0.644531,1008.0,210.875,5.3125,17,0


In [29]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20216100 entries, 0 to 20216099
Data columns (total 30 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   building_id                    int16  
 1   meter                          int8   
 2   hour                           int64  
 3   weekend                        int64  
 4   month                          int64  
 5   dayofweek                      int64  
 6   site_id                        int8   
 7   primary_use                    int8   
 8   square_feet                    int32  
 9   year_built                     int16  
 10  floor_count                    int16  
 11  air_temperature                float16
 12  cloud_coverage                 int16  
 13  dew_temperature                float16
 14  air_temperature_mean_lag3      float16
 15  cloud_coverage_mean_lag3       float16
 16  dew_temperature_mean_lag3      float16
 17  precip_depth_1_hr_mean_lag3    float16
 18  

In [30]:
import gc

In [31]:
gc.collect()

40

In [34]:
from typing import Tuple

In [35]:
def rmsle(predt: np.ndarray, dtrain: xgb.DMatrix) -> Tuple[str, float]:
    ''' Root mean squared log error metric.'''
    y = dtrain.get_label()
    predt[predt < -1] = -1 + 1e-6
    elements = np.power(np.log1p(y) - np.log1p(predt), 2)
    return 'PyRMSLE', float(np.sqrt(np.sum(elements) / len(y)))

In [36]:

%%time

folds = 3
seed = 99 #666
import pickle
kf = KFold(n_splits=folds, shuffle=False, random_state=seed)
scores = [] 
models = []
for i,(train_index, val_index) in enumerate(kf.split(train_df)):
    train_X = train_df.iloc[train_index]
    val_X = train_df.iloc[val_index]
    train_y = target.iloc[train_index]
    val_y = target.iloc[val_index]

    xgb_train = xgb.DMatrix(train_X, train_y)
    xgb_eval = xgb.DMatrix(val_X, val_y)

    gbm = xgb.train(xgb_params, xgb_train, num_boost_round = 3000,evals=[(xgb_train, 'train'), (xgb_eval, 'val')],
                    feval=rmsle, verbose_eval = 30 , early_stopping_rounds = 30)

    # save
    pickle.dump(gbm,open('xgb_%s.pickle'%i, "wb") )
    gbm.__del__() 
    print('delete')
#     print(gbm)
    del xgb_train, xgb_eval
    gc.collect()

for i in range(folds):
    models.append(pickle.load(open("xgb_%s.pickle"%i, "rb")))

C:\Users\user\anaconda3\envs\tftorch\lib\site-packages\sklearn\model_selection\_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


[21:39:18] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { missing, reg_lamdba } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:4.09005	val-rmse:3.95362	train-PyRMSLE:1.14736	val-PyRMSLE:1.11631
Multiple eval metrics have been passed: 'val-PyRMSLE' will be used for early stopping.

Will train until val-PyRMSLE hasn't improved in 30 rounds.
[30]	train-rmse:1.55025	val-rmse:1.66946	train-PyRMSLE:0.42149	val-PyRMSLE:0.49799
[60]	train-rmse:1.13602	val-rmse:1.35270	train-PyRMSLE:0.37851	val-PyRMSLE:0.47770
[90]	train-rmse:1.01738	val-rmse:1.27493	train-PyRMSLE:0.35605	val-PyRMSLE:0.46328
[120]	train-rmse:0.95067	val-rmse:1.22815	train-PyRMSLE:0.34165	val-PyRMSLE:0.44990
[150]	train-rmse:0.91127	val-rmse:1.20196

In [37]:
%%time
i = 0
res = []
res2 = []
res3 = []
step_size = 50000
for j in tqdm(range(int(np.ceil(test_df.shape[0] / 50000)))):
    res2.append(sum([np.expm1(model.predict(xgb.DMatrix(test_df.iloc[i:i + step_size]))) for model in models])/ folds)
    
    i += step_size

100%|████████████████████████████████████████████████████████████████████████████████| 834/834 [09:22<00:00,  1.48it/s]

Wall time: 9min 22s


In [38]:
%%time
from datetime import datetime
sample_submission = pd.read_csv('./data/sample_submission.csv')
res2 = np.concatenate(res2)
sample_submission["meter_reading"] = res2
sample_submission.loc[sample_submission['meter_reading'] < 0, 'meter_reading'] = 0
sample_submission.to_csv('xgb_exp_avg_sub_' + str(datetime.now().strftime('%Y-%m-%d_%H-%M-%S')) + '.csv', index=False)
sample_submission.head(10)

Wall time: 1min 9s


,row_id,meter_reading
0,0,0.976197
1,1,0.661841
2,2,0.273727
3,3,1.581491
4,4,2.914666
5,5,0.371887
6,6,1.913357
7,7,3.459800
8,8,93.599487
9,9,1.496152
